In [1]:
from manim import *
import rc_lib

from rc_lib import types as T
from rc_lib import style
from rc_lib.math_utils import geometry
from rc_lib.design_utils import plate
from rc_lib.view_utils import title_sequence

quality = "ql"

Manim Community v0.17.2

AttributeError: module 'enum' has no attribute 'StrEnum'

The reccomended way to create plates is by first drawing your holes and other relevant geometry, marking out the outer boundary using a second set of circles, and then connecting the boundary with tangent lines.

In [ ]:
class IntakePlateScene(Scene):
    def construct(self):
        small_base = plate.PlateCircle.make(0.15, 0.2)
        medium_base = plate.PlateCircle.make(0.4, 0.2)

        front_hole = geometry.point_2d(-4, -3)
        middle_hole = geometry.point_2d(-1.5, 0.25)
        back_hole = geometry.point_2d(2.5, 1.5)

        back_offset = geometry.point_2d(0.8, 0.75)

        points = [
            medium_base.copy(front_hole),
            medium_base.copy(middle_hole),
            medium_base.copy(back_hole),
            small_base.copy(back_hole + back_offset),
            small_base.copy(back_hole + geometry.point_2d(1, -0.2)),
            small_base.copy((middle_hole + back_hole) / 2),
            small_base.copy((front_hole + middle_hole) / 2),
        ]

        boundary_order = [1, 3, 4, 0]
        plate_group = plate.PlateGroup(points, boundary_order)
        title = title_sequence.TitleSequence(
            ["Draw plate holes", "Add larger circles", "Connect boundary"])

        self.play(title.next(), run_time=style.Time.FAST)
        self.play(plate_group.draw_inner_circles())
        self.wait(style.Time.STANDARD)

        self.play(title.next(), run_time=style.Time.FAST)
        self.play(plate_group.draw_outer_circles())
        self.wait(style.Time.STANDARD)

        self.play(title.next(), run_time=style.Time.FAST)
        self.play(plate_group.draw_boundary(), run_time=style.Time.SLOW)
        self.wait(style.Time.END)

In the event a hole is too close to the edge of a plate, you may need to redraw the boundary.

In [ ]:
%%manim -v WARNING $quality BoundaryRedrawScene

class BoundaryRedrawScene(Scene):
    def construct(self):
        left = PlateCircle(1.75, 0.75)
        right = PlateCircle(1.75, 0.75)
        left.move_to([-6, -2, 0])
        right.move_to([6, -2, 0])
        self.add(left)
        self.add(right)

        line = PlateCircle.tangent_line(right, left)
        self.add(line)

        middle = PlateCircle(1, 0.75)
        middle.move_to([0, -.75, 0])
        self.add(middle.inner_circle())

        title = Text("1. Add outer circle", font_size=MED_FONT)
        title.to_corner(LEFT + UP)
        self.play(Write(title))
        self.play(middle.draw_outer_circle())

        title_2 = Text("2. Redraw boundary", font_size=MED_FONT)
        title_2.to_corner(LEFT + UP)
        self.play(Transform(title, title_2))

        self.play(Uncreate(line))
        self.play(Create(PlateCircle.tangent_line(right, middle)))
        self.play(Create(PlateCircle.tangent_line(middle, left)))
        self.wait(3)

In [ ]:
%%manim -v WARNING -$quality LineConstraintScene

class LineConstraintScene(Scene):
    def construct(self):
        left = PlateCircle(1.75, 0.75)
        right = PlateCircle(1.75, 0.75)
        left.move_to([-6, -2, 0])
        right.move_to([6, -2, 0])
        self.add(left)
        self.add(right)

        tangent_points = PlateCircle.tangent_points(right, left)

        title = Text("1. Create line", font_size=MED_FONT)
        title.to_corner(LEFT + UP)
        self.play(Write(title))

        left_start_point = tangent_points[1] + [1.75, 0.75, 0]
        right_start_point = tangent_points[0] + [-2, 0.5, 0]
        line = LineSegment(left_start_point, right_start_point)

        self.play(line.create())

        title_2 = Text("2. Add coincident constraints", font_size=MED_FONT)
        title_2.to_corner(LEFT + UP)
        self.play(Transform(title, title_2))

        left_coincident_point = left.center() + normalize(left_start_point - left.center()) * left.outer_radius()
        self.flash(line.start_point(), left)
        self.play(line.move_start(left_coincident_point))

        right_coincident_point = right.center() + normalize(right_start_point - right.center()) * right.outer_radius()
        self.flash(line.end_point(), right)
        self.play(line.move_end(right_coincident_point))


        title_3 = Text("3. Add tangent constraints", font_size=MED_FONT)
        title_3.to_corner(LEFT + UP)
        self.play(Transform(title, title_3))

        self.flash(line.start_point(), left)
        angle = angle_between_vectors(left_coincident_point - left.center(), tangent_points[1] - left.center())
        self.play(line.move_start(tangent_points[1]), path_arc=angle, path_arg_centers=[left.center()])

        self.flash(line.end_point(), right)
        angle = -angle_between_vectors(right_coincident_point - right.center(), tangent_points[0] - right.center())
        self.play(line.move_end(tangent_points[0]), path_arc=angle, path_arg_centers=[right.center()])

        self.wait(3)
    
    def flash(self, point, circle):
        self.play(Flash(point), run_time=0.75)
        self.play(Flash(circle, flash_radius=circle.outer_radius(), num_lines=40), run_time=0.75)
